# Introduction - Retrieval Augmented Generation using `Weaviate` and `SK`

Implements a simple workflow of retrieve then generate. Using `semantic kernel` to help with prompt engineering and orchestrating interactions with LLMs and Weaviate as a knowledge-base from which to retreive semantically relevant context. 

This allows us to not only control what is being generated but also cite sources along with the relevance of those sources.

## Setup

In [ ]:
#!pip install semantic-kernel
#!pip install weaviate-client
#!pip install python-dotenv

## OS-specific notes:
* if you run into SSL errors when connecting to OpenAI on macOS, see this issue for a [potential solution](https://github.com/microsoft/semantic-kernel/issues/627#issuecomment-1580912248)
* on Windows, you may need to run Docker Desktop as administrator

In [1]:
from typing import Tuple
import os
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    OpenAITextCompletion,
    OpenAITextEmbedding,
)

In [2]:
from semantic_kernel.connectors.memory.weaviate import weaviate_memory_store
from dotenv import load_dotenv

load_dotenv(override=True)

# Using a locally deployed instance of Weaviate 
config = weaviate_memory_store.WeaviateConfig(url="http://localhost:8080")

store = weaviate_memory_store.WeaviateMemoryStore(config=config)

#THE LINE BELOW WILL DELETE ALL DATA IN YOUR SCHEMA!
store.client.schema.delete_all()

Then, we register the memory store to the kernel:

In [3]:
kernel = sk.Kernel()

api_key, org_id = sk.openai_settings_from_dot_env()

kernel.add_text_completion_service(
    "dv", OpenAITextCompletion("text-davinci-003", api_key, org_id)
)
kernel.add_text_embedding_generation_service(
    "ada", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id)
)

kernel.register_memory_store(memory_store=store)
kernel.import_skill(sk.core_skills.TextMemorySkill())

{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x12affab10>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x12affbe10>}

## First with no RAG, just Generate:

In [4]:
# Create a reusable function with one input parameter

prompt = "You are a friendly and talkative AI. Answer {{$input}}"

answer = kernel.create_semantic_function(prompt)

print(answer("Do I have pets?"))



That depends on you! Do you have any pets?


### Add some prompt engineering:

In [6]:
prompt = """You are a friendly and talkative AI. Answer {{$input}} or say 
            'I don't know.' if you don't have an answer.
         """

answer2 = kernel.create_semantic_function(prompt)

print(answer2("Do I have pets?"))


I don't know.


## Manually adding memories


Let's create some initial memories "About Me". We can add memories to our weaviate memory store by using `save_information_async`

In [10]:
COLLECTION = "AboutMe"

In [11]:
#Function for persisting a memory to Weaviate
import uuid

async def populate_memory(kernel: sk.Kernel) -> None:
    # Add some documents to the semantic memory
    await kernel.memory.save_information_async(COLLECTION, 
                                               id = str(uuid.uuid4()), 
                                               text = 'When I turned 5 my parents gifted me goldfish for my birthday')
    
    await kernel.memory.save_information_async(COLLECTION,
                                              id = str(uuid.uuid4()),
                                              text = 'I love datascience')
    
    await kernel.memory.save_information_async(COLLECTION,
                                              id = str(uuid.uuid4()),
                                              text = 'I have a black nissan sentra')
    
    await kernel.memory.save_information_async(COLLECTION,
                                              id = str(uuid.uuid4()),
                                              text = 'my favourite food is popcorn')
    
    await kernel.memory.save_information_async(COLLECTION,
                                              id = str(uuid.uuid4()),
                                              text = 'I like to take long walks.')
    print("Sucessfully populated memories!")

In [12]:
await populate_memory(kernel)

Sucessfully populated memories!


In [14]:
#Conduct semantic search

result = await kernel.memory.search_async(COLLECTION, 'Do I have a pet?')
print(f"Retreived document: {result[0].text}")

Retreived document: When I turned 5 my parents gifted me goldfish for my birthday


In [15]:
result[0].relevance

0.891325980424881

In [16]:
async def setup_RAG(kernel: sk.Kernel) -> Tuple[sk.SKFunctionBase, sk.SKContext]:
    sk_prompt = """
    You are a friendly and talkative AI.
    
    Answer the {{$user_input}}. You may use information provided in {{$retreived_context}} it may be useful. 
    Say 'I don't know.' if you don't have an answer. 
    """.strip()

    rag_func = kernel.create_semantic_function(sk_prompt, max_tokens=200, temperature=0.8)

    context = kernel.create_new_context()

    return rag_func, context

In [18]:
async def RAG(kernel: sk.Kernel, rag_func: sk.SKFunctionBase, context: sk.SKContext) -> bool:

    user_input = input("User:> ")
    context["user_input"] = user_input

    #Retrieve
    result = await kernel.memory.search_async(COLLECTION,context["user_input"])
        
    # Can use more ie. upto k retrieved documents by: kernel.memory.search_async(COLLECTION,limit=k,context["user_input"])
    if result[0].text:
        context["retreived_context"] = result[0].text
    
    #Then generate
    answer = await kernel.run_async(rag_func, input_vars=context.variables)

    print(f"\n\u001b[34mChatBot:> {answer}\u001b[0m \n\n\033[1;32m Source: {context['retreived_context']}\n Relevance: {result[0].relevance}\u001b[0m \n")
    return False

# Q&A powered by RAG!

In [19]:
print("Setup Q&A with memory!")
rag_func, context = await setup_RAG(kernel)

print("Ask a question (type 'exit' to exit):\n")
answer = await RAG(kernel, rag_func, context)

Setup Q&A with memory!
Ask a question (type 'exit' to exit):

User:> What did my parents get me?

ChatBot:> 

It sounds like your parents got you a goldfish for your birthday when you were 5! 

 Source: When I turned 5 my parents gifted me goldfish for my birthday
 Relevance: 0.9213910102844238 

